# Children Act applications csv

### This code summarises the information produced from the children act applications code into tidy data sets, and combines them all at the end.
1. Children by order type
2. Individual children (annual)
3. Individual children (quarterly)
4. Orders applied for
5. Applications (events)
6. Case starts (including applicants/respondents and high court indicator)

In [ ]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [ ]:
# define some variables to be used throughout the notebook
db2 = "fcsq"

In [ ]:
#Applications by type - children involved
pydb.create_temp_table( 
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc AS Order_type,
   Order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_child_d
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  order_case_type,
  Order_desc,
  Order_code
""",

"ca_apps_type_child")

In [ ]:
#Individual children - annual - including age of child
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(NULL AS VARCHAR) AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_yr_d
Where
  Year > 2010
Group BY
  Year,
  order_case_type,
  Gender,
  age_band
""",

"ca_apps_ind_yr")

In [ ]:
#Individual children - quarterly - including age of child
pydb.create_temp_table(
f"""
SELECT
  CAST(NULL AS INTEGER) AS Year,
  CONCAT(CAST(year AS VARCHAR),'-',CAST(quarter AS VARCHAR)) AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_qtr_d
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  order_case_type,
  Gender,
  age_band
""",

"ca_apps_ind_qtr")

In [ ]:
#Orders applied for (by order type)
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Order type' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc,
   Order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_order_types_d
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  order_case_type,
  Order_desc,
  Order_code
""",

"ca_apps_type_order")

In [ ]:
#Application events
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Application events' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_event_count_d
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  order_case_type
""",

"ca_apps_events")

In [ ]:
#Case starts - including number of applicants/respondents and Hight court indicator*/
pydb.create_temp_table(
f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Cases' AS Type,
  'Cases starting' AS Count_type,
  CASE WHEN main_case_type = 'C' THEN 'Public law'
       WHEN main_case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
   CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
   Case When no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END AS Applicants_in_case,
   Case When no_of_respondents in (1,2,3)
         THEN CAST(no_of_respondents AS VARCHAR)
         ELSE '4+'
        END AS Respondents_in_case,
   CASE WHEN case_hc_value = 'N' THEN 'No'
          ELSE HC_London_Ind
     END AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_case_HC_d
WHERE
  Year > 2010
Group BY
  Year,
  Quarter,
  main_case_type,
  Case When no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END,
  Case When no_of_respondents in (1,2,3)
         THEN CAST(no_of_respondents AS VARCHAR)
         ELSE '4+'
        END,      
  CASE WHEN case_hc_value = 'N' THEN 'No'
          ELSE HC_London_Ind
     END
""",

"ca_apps_cases")

In [ ]:
#Union together datasets
pydb.create_temp_table(
f"""
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_CHILD
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_YR
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_QTR
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_ORDER
UNION ALL
SELECT
  *
FROM
 __temp__.CA_APPS_EVENTS
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_CASES

""",
    
"ca_apps_csv_d")

### Load to dataframe and put in S3

In [ ]:
data = pydb.read_sql_query ("select * from __temp__.ca_apps_csv_d")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv ('Exports/apps_data_with_no_child_case_type_update.csv', header = True)

In [ ]:
df.to_csv (r's3://alpha-family-data/fcsq_processing/CA_apps/test_csv/ca_apps_d.csv', header = True)